# Spatio-temporal analysis of the actin fusion focus component
### Compute sets of parameters from aligned trajectorieds

Should be used after Method1.2 notebook. 

#### Package required:
-trajalign from Dr. A.Picco (https://apicco.github.io/trajectory_alignment/)

-numpy

-matplotlib

#### Plots produced: 
-

#### Inputs required:
-path: path to the folder containing the POI subfolders

-files: list of POI subfolders names

-colPOI: list of colors associated to each POI for plotting
-for each plot: the filename under which it is saved (from cells under 'Plot subtitle')

In [ ]:
path='/pathtoanalysisfolder' #folder containing the POI subfolders on which to run the alignment
folder='/aligned_trajectories/Data/average/' #follow up path to median parameters, if the pipeline is used this should not be changed
files =['POI1','POI2']  # list of POI subfolders to treat
colPOI=['color1','color2'] # list of each POI colors for plotting
n=len(files)

In [ ]:
#import python packages
from trajalign.traj import Traj
from trajalign.average import load_directory
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm
import os

In [ ]:
#plot the .f() component of median trajectories files, the what parameters correspond to the type of data stored in .f()
# possible what parameters: 'f' (median fluorescence) , 'distP'(distance from map reference), 'distPM' (distance between references)
# as of now, plot parameters such x and y limits can be changed from inside the function

def plot_f_info (list, color, names, sample='Rep', what='f'):
    
    if sample=='tot':
        list=[item for sublist in list for item in sublist]
        ax=plt.subplot(111)
    a=0.1                    # base opacity, can be changed to make curves more opaque
    for i in range(len(list)):
        x=list[i].t()
        y=list[i].f()
        sd_u=list[i].f()+list[i].f_err()
        sd_l=list[i].f()-list[i].f_err()
        if sample=='tot':
            col=color[i]
            label=names[i]
        elif sample == 'Rep':
            label='Rep_'+str(i+1)
            col=color
            
        plt.plot(x,y,color=col, alpha =(a+0.4), label=label, linewidth=8)
        plt.scatter(x,y,color=col, alpha =(a+0.4),s=36)
        if sample == 'Rep':
            plt.fill_between(x,sd_u,sd_l,color=col, alpha =a )
            a+=0.01
        elif sample=='tot':
            plt.fill_between(x,sd_u,sd_l,color=col, alpha =a )
            plt.plot(x,sd_u,color=col,alpha=0.4,linestyle=':')
            plt.plot(x,sd_l,color=col,alpha=0.4,linestyle=':')
            
        plt.xlim(-100,15)
        plt.xlabel ('time from fusion (min)',fontsize=28)
        plt.tight_layout()
        if sample=='Rep':
            plt.title(names,fontsize=32) 
        handles, labels = plt.gca().get_legend_handles_labels()
        labels, ids = np.unique(labels, return_index=True) 
        handles = [handles[i] for i in ids] 
        if sample=='tot':
            box = ax.get_position()
            plt.legend(handles, labels, loc='center right',fontsize=28,bbox_to_anchor=(1.15, 0.5)) 
        else:
            plt.legend(handles, labels, loc='best',fontsize=28) 
        plt.grid(visible=True)
        plt.xticks(fontsize=26)
        plt.yticks(fontsize=26)
        
        if what=='f':
            plt.ylim(-0.1,1.1)
            plt.ylabel ('median fluorescent int. (AU)',fontsize=28)
            plt.vlines(0,-0.1,1.1, color='black',linewidth=4)
        
        elif what=='distP':
            plt.ylim(-150,150)
            plt.ylabel ('median distance from reference (nm)',fontsize=28)
            plt.hlines(0,-60,15, color='red',linewidth=4)
            
        elif what=='distPM':
            plt.ylim(-10,800)
            plt.ylabel ('median distance from Myo_P (nm)',fontsize=28)
            plt.hlines(0,-60,15, color='red',linewidth=4)

In [ ]:
#Load datasets
Tot_av_POI=[[] for _ in range(n)]
Tot_av_Myo_P=[[] for _ in range(n)]
Tot_av_Myo_M=[[] for _ in range(n)]

Tot_av_distP_POI=[[] for _ in range(n)]
Tot_av_distP_M=[[] for _ in range(n)]
Tot_av_distM_POI=[[] for _ in range(n)]
 
for i in range (n):
    POI= load_directory(path+files[i] + folder,pattern= 'coordinates_POI.txt', comment_char = '#', t_unit='min' ,t= 1, coord = ( 2 ,3 ) , f = 4 ,coord_err=(5 ,6),f_err = 7 ,coord_unit = 'nm' )
    Tot_av_POI[i].extend(POI)
    MP= load_directory(path+files[i] +folder,pattern= 'coordinates_Myo_P_.txt', comment_char = '#', t_unit='min' ,t= 1, coord = ( 2 ,3 ) , f = 4 ,coord_err=(5 ,6),f_err = 7 ,coord_unit = 'nm' )
    Tot_av_Myo_P[i].extend(MP)
    MM= load_directory(path+files[i] +folder,pattern= 'coordinates_Myo_M.txt', comment_char = '#', t_unit='min' ,t= 1, coord = ( 2 ,3 ) , f = 4 ,coord_err=(5 ,6),f_err = 7 ,coord_unit = 'nm' )
    Tot_av_Myo_M[i].extend(MM)
    
    dir_POI= load_directory(path+files[i] +folder,pattern= 'P_POI.txt', comment_char = '#', t_unit='min' ,t= 0 , f = 1,f_err = 2 ,f_unit= 'nm' )
    Tot_av_distP_POI[i].extend(dir_POI)
    dir_MP= load_directory(path+files[i] + folder,pattern= 'P_M.txt', comment_char = '#', t_unit='min' ,t= 0 , f = 1,f_err = 2 ,f_unit= 'nm' )
    Tot_av_distP_M[i].extend(dir_MP)
    dir_MM= load_directory(path+files[i] +folder,pattern= 'M_POI.txt', comment_char = '#', t_unit='min' ,t= 0 , f = 1,f_err = 2 ,f_unit= 'nm' )
    Tot_av_distM_POI[i].extend(dir_MM)

### Plots
Plot will be saved in the analysis folder

In [ ]:
# Plot median distance from reference
plt.figure(figsize=(25,10)) 
plot_f_info(Tot_av_distP_POI,colPOI,files,sample="tot",what='distP')
figname=path+'/filename.pdf'     
plt.savefig(figname)
plt.show()
plt.close()

In [ ]:
# Plot median fluorescence
plt.figure(figsize=(30,20))
plot_f_info(Tot_av_POI,colPOI,files,sample="tot",what='f')
figname=path+'/filename.pdf'
plt.savefig(figname)
plt.show()
plt.close()